# Tabular Playground Series - Oct 21

This month, our data consists of 284 feature variables and our target variable is binary classification. We will first perform some basic EDA to take a better look at this data following which we will start working on our models. 

## Plan

Moving forward this is the plan we are going to be following. Keep in mind, this is not a concrete plan and I might change it as we move through the notebook. This will show you my process on how I approach these datasets.

- *Memory Reduction*
- *Sampling to Reduce Training Time*
- *EDA*
- *Model Development*
- *Hyperparameter Tuning*
- *Feature Importance from top models*
- *Selecting the best Model*

## Imports 

Let's import some of the libraries we will be using throughout the notebook

In [2]:
# Data Import on Kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Importing processing libraries
import numpy as np
import pandas as pd

# Importing Visualisation libraries
import seaborn as sns
import matplotlib.pyplot as plt

# Importing libraries for the metrics
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

# Importing libraries for the model
import xgboost as xgb 
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor


In [3]:
data = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')

## Memory Reduction

Here, we will take a look at the memory consumption by the current data and each feature following which we will try to reduce it to some extent. 

In [4]:
memory_usage = data.memory_usage(deep=True) / 1024 ** 2
print('memory usage of features: \n', memory_usage.head(7))
print('memory usage sum: ',memory_usage.sum())

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

reduced_df = reduce_memory_usage(data, verbose=True)

In [ ]:
reduced_df.describe()

## Sampling Data

Now that we have reduced the memory usage by over 70%, let's sample the data. We are doing this to reduce the model training time. The sampling would preserve the distributions of each feature while taking only 20% of the entire dataset. We can then perform EDA, modelling, hyperparameter tuning and other steps on this sampled data.

Once we decide on the model we want to use, we can train the final model on the entire dataset again.

In [ ]:
sample_df = reduced_df.sample(int(len(reduced_df) * 0.2))
sample_df.shape

sample_df = sample_df.drop(['id'], axis=1)

In [ ]:
# Let's confirm if the sampling is retaining the feature distributions

fig, ax = plt.subplots(figsize=(6, 4))

sns.histplot(
    data=reduced_df, x="f6", label="Original data", color="red", alpha=0.3, bins=15
)
sns.histplot(
    data=sample_df, x="f6", label="Sample data", color="green", alpha=0.3, bins=15
)

plt.legend()
plt.show();

## EDA

Let's start looking at any correlations that might exist among the features.
We will also be looking at the densities of every feature.

In [ ]:
sample_df

In [ ]:
# Check na values
print('Amount of existing NaN values', sample_df.is_na().sum())

# Target Class Distribution
print('Amount of existing NaN values', sample_df.is_na().sum())

In [ ]:
f, ax = plt.subplots(figsize=(8, 6))
corr = sample_df.iloc[:,:20].corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)
plt.show()

Before we look at distributions, we need to split the data into continuous and categorical variables.

In [ ]:
cat_variables = []

for column in sample_df.columns:
    if len(sample_df[column].unique()) < 10:
        cat_variables.append(column)
print(cat_variables)

So, the categorical features in the dataset are f22, f43 and all the ones from f242-f284. Let's find the  distributions of the rest using kdeplot and the distributions of these using barplot (compared to target).

In [ ]:
fig = plt.figure(figsize = (18, 100))

for i in range(len(sample_df.columns.tolist()[:241])):
    if sample_df.columns.tolist()[:241][i] in ['f22', 'f43']: 
        continue
    else:
        plt.subplot(25,10,i+1)
        sns.set_style("white")
        plt.title(sample_df.columns.tolist()[:241][i], size = 12, fontname = 'monospace')
        a = sns.kdeplot(sample_df[sample_df.columns.tolist()[:241][i]], color = '#1a5d57', shade = True, alpha = 0.9, linewidth = 1.5, edgecolor = 'black')
        plt.ylabel('')
        plt.xlabel('')
        plt.xticks(fontname = 'monospace')
        plt.yticks([])
        for j in ['right', 'left', 'top']:
            a.spines[j].set_visible(False)
            a.spines['bottom'].set_linewidth(1.2)
        
fig.tight_layout(h_pad = 3)

plt.show()

In [ ]:
# Code from https://www.kaggle.com/craigmthomas/tps-oct-2021-eda

cat_features = ["f22", "f43"]
cat_features.extend(["f{}".format(x) for x in range(242, 285)])

fig, axs = plt.subplots(11, 4, figsize=(4*4, 11*3), squeeze=False, sharey=True)

ptr = 0
for row in range(11):
    for col in range(4):  
        x = train[[cat_features[ptr], "target"]].value_counts().sort_index().to_frame().rename({0: "# of Samples"}, axis="columns").reset_index()
        sns.barplot(x=cat_features[ptr], y="# of Samples", hue="target", data=x, ax=axs[row][col], **sns_params)
        plt.xlabel(cat_features[ptr])
        ptr += 1
        del(x)
plt.tight_layout()    
plt.show()

_ = gc.collect()